In [78]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from skimage import feature, exposure
from matplotlib.pyplot import bar
from skimage.transform import resize
import sklearn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from PIL import Image, ImageEnhance
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.calibration import CalibratedClassifierCV
from sklearn.neural_network import MLPClassifier
import os

In [50]:
#https://scikit-image.org/docs/dev/auto_examples/features_detection/plot_local_binary_pattern.html
#https://cvexplained.wordpress.com/2020/07/22/10-7-local-binary-patterns/
#numPoints=24, radius=8
def LocalBinaryPatterns(image, eps=1e-7,numPoints=16, radius=2):
    # compute the Local Binary Pattern representation
        # of the image, and then use the LBP representation
        # to build the histogram of patterns
        #https://scikit-image.org/docs/dev/api/skimage.feature.html#local-binary-pattern
        #'uniform', the result only includes patterns where all black dots are adjacent and all white dots are adjacent. All other combinations are labeled 'non-uniform'.
        #Non-uniform results: If you look at your result more closely, there are actually 243 bins, not 242. The 243th bin lumps together all non-uniform results.
        #image = image.astype('uint8')
        #image = np.asarray(image, dtype=np.uint8)
        #image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        lbp = feature.local_binary_pattern(image, numPoints,radius, method='nri_uniform')
        pinnum=lbp.ravel().max()
        (hist, edges) = np.histogram(lbp.ravel(),bins=np.arange(0,pinnum))
        # normalize the histogram
        hist = hist.astype("float")
        hist /= (hist.sum() + eps)
        # return the histogram of Local Binary Patterns
        return hist

In [51]:
lpb_detected_female = []
for dirname, _, filenames in os.walk('..//input//cmp23-handwritten-males-vs-females//Females//Females//'):
    for filename in filenames:
        img = cv2.imread(os.path.join(dirname,filename),0)
        if img is not None:
            lpb_detected_female.append(LocalBinaryPatterns(img))
lpb_detected_female = np.array(lpb_detected_female)

lpb_detected_male = []
for dirname, _, filenames in os.walk('..//input//cmp23-handwritten-males-vs-females//Males//Males//'):
    for filename in filenames:
        img = cv2.imread(os.path.join(dirname,filename),0)
        if img is not None:
            lpb_detected_male.append(LocalBinaryPatterns(img))
lpb_detected_male = np.array(lpb_detected_male)

In [54]:
# classifier
X=np.concatenate((lpb_detected_male,lpb_detected_female),axis=0)

Y=np.concatenate((np.ones(lpb_detected_male.shape[0]),
                np.zeros(lpb_detected_female.shape[0])),
                axis=0)

In [55]:
X,Y=sklearn.utils.shuffle(X,Y)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2,random_state=109) # 70% training and 30% test

In [67]:
svm = LinearSVC(C=100.0, random_state=42)
model = CalibratedClassifierCV(svm)
model.fit(X_train, y_train)

In [68]:
y_pred = model.predict(X_test)
# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [70]:
model.probability=False
prob = model.predict_proba(X_test)

In [74]:
model2 = RandomForestClassifier(n_estimators = 500, max_features = 'sqrt', criterion = 'gini', n_jobs=-1)

# Train the model using the training sets
model2.fit(X_train, y_train)

In [76]:
y_pred = model2.predict(X_test)
# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

predictions = model2.predict_proba(X_test)
print(predictions)

In [73]:
model3 = KNeighborsClassifier(n_neighbors=5)

# Train the model using the training sets
model3.fit(X_train, y_train)

y_pred = model3.predict(X_test)
# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [79]:
model4 = MLPClassifier(solver='lbfgs', 
                    alpha=1e-5,
                    hidden_layer_sizes=(2,), 
                    random_state=1)

model4.fit(X_train, y_train) 
y_pred = model4.predict(X_test)
# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))